# Turner Sandstone Production Prediction
Last week we looked at data wrangling to get our data organized and ready for machine learning. We now have a full `DataFrame` with values for each feature for each well. First we are going to import pandas to read in our munged data, and a few functions from scikit-learn to do the heavy lifting for the machine learning portion of the notebook. We are going to use `RandomForestRegressor` to find out which features are most important for predicting the first 18 months of oil production. We will use `train_test_split` to split our data into training and test sets to validate our model, and `RandomizedSearchCV` will be used to find the best hyperparameters for our random forest regressor. Let's get started!

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV

Let's go ahead and read in our munged data from the `.csv` file

In [2]:
munged_data = pd.read_csv("organized_turner_data.csv")

Let's start by setting the value we want to predict (production) to a `numpy array`

In [3]:
y = munged_data["First 18 months oil (bbl)"].values

Next let's remove the production values from our `dataframe`

In [4]:
X = munged_data.drop(["First 18 months oil (bbl)"], axis=1)

Then we split the data into training and test datasets. This way the model won't see all the data during the training phase and we can validate on the test data

In [5]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.01, random_state=19
)

Next we set up a random grid of hyperparameters for `RandomizedSearchCV` to wander through to find the best predictive model. A full description is found in the scikit-learn docs at https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html

In [6]:
# the number of trees in the forest
n_estimators = [int(x) for x in np.linspace(start=100, stop=900, num=100)]
# the number of features to use at each split
max_features = ["auto", "sqrt"]
# max number of levels in each tree
max_depth = [int(x) for x in np.linspace(10, 220, num=11)]
max_depth.append(None)
# minimum samples needed to split a tree
min_samples_split = [2, 5, 10, 15, 20]
# minimum samples required at each leaf node
min_samples_leaf = [1, 2, 4, 8, 16]
# method for selecting samples
bootstrap = [True, False]

# create the grid
random_grid = {
    "n_estimators": n_estimators,
    "max_features": max_features,
    "max_depth": max_depth,
    "min_samples_split": min_samples_split,
    "min_samples_leaf": min_samples_leaf,
    "bootstrap": bootstrap,
}

Next we will fit our `RandomForestRegressor` to the training data, and use 3 fold cross validation for 100 iterations. This finds the best hyperparameters for our model

In [7]:
rf = RandomForestRegressor()
rf_random = RandomizedSearchCV(
    estimator=rf,
    param_distributions=random_grid,
    n_iter=100,
    cv=3,
    verbose=2,
    random_state=19,
    n_jobs=-1,
)
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    8.6s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   32.1s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  1.1min finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
          fit_params=None, iid='warn', n_iter=100, n_jobs=-1,
          param_distributions={'n_estimators': [100, 108, 116, 124, 132, 140, 148, 156, 164, 172, 180, 188, 196, 205, 213, 221, 229, 237, 245, 253, 261, 269, 277, 285, 293, 302, 310, 318, 326, 334, 342, 350, 358, 366, 374, 382, 390, 398, 407, 415, 423, 431, 439, 447, 455, 463, 471, 479, 487, 495, 504, 512, 52...amples_split': [2, 5, 10, 15, 20], 'min_samples_leaf': [1, 2, 4, 8, 16], 'bootstrap': [True, False]},
          pre_dispatch='2*n_jobs', random_state=19, re

These are the best fit parameters

In [8]:
rf_random.best_params_

{'n_estimators': 196,
 'min_samples_split': 2,
 'min_samples_leaf': 2,
 'max_features': 'sqrt',
 'max_depth': 178,
 'bootstrap': False}

How much better are the best fit parameters compared to the standard 'out of the box' version of the `RandomForestRegressor`

In [9]:
def compare(model, test_features, test_actual):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_actual)
    mape = 100 * np.mean(errors / test_actual)
    accuracy = 100 - mape
    print("Model Performance")
    print("Average Error: {:0.4f} barrels".format(np.mean(errors)))
    print("Accuracy = {:0.2f}%.".format(accuracy))

    return accuracy


base_model = RandomForestRegressor(n_estimators=10, random_state=42)
base_model.fit(X_train, y_train)
base_accuracy = compare(base_model, X_train, y_train)

best_random = rf_random.best_estimator_
random_accuracy = compare(best_random, X_train, y_train)

Model Performance
Average Error: 13141.2128 barrels
Accuracy = 51.45%.
Model Performance
Average Error: 5187.3445 barrels
Accuracy = 80.99%.


In [10]:
print(
    "The model is {:0.2f}% accurate on the test data".format(
        100 * best_random.score(X_test, y_test)
    )
)  # let's check the model on the test data

The model is 74.47% accurate on the test data


So our model does ok on the test data, let's use the `feature_importances_` of the `best_random` model to see which features are most important in predicting the first 18 months of oil production.

In [11]:
feat_labels = X.columns.values  # get the feature labels
feature = list(
    zip(feat_labels, best_random.feature_importances_)
)  # make a list of the feature labels and the importance values
sorted(
    feature, key=lambda tup: tup[1], reverse=True
)  # sort from most to least important feature in predicting production

[('Total vertical depth at bottom hole (ft)', 0.11960542352545994),
 ('Surface hole longitude (NAD83)', 0.0809194686492954),
 ('Bottom hole longitude (NAD83)', 0.07627055885098187),
 ('Average gas-oil ratio (ft3/bbl)', 0.07529668554350513),
 ('Total proppant (lb)', 0.07128679299929348),
 ('Surface-to-bottom hole length (ft)', 0.06390807852259404),
 ('APINO', 0.05245075239719613),
 ('Unnamed: 0', 0.051164358081107676),
 ('Number of frac stages', 0.05073672963814896),
 ('Producing interval length (ft)', 0.050415724278201346),
 ('IP oil API gravity (°)', 0.048883895694935495),
 ('API number', 0.047979678905179915),
 ('Bottom hole latitude (NAD83)', 0.04744873764261307),
 ('Surface hole latitude (NAD83)', 0.04461134548586631),
 ('Lateral azimuth (°)', 0.041852140214180794),
 ('Total slurry (bbl)', 0.03755086066764833),
 ('Calculated top of reservoir temperature (°F)', 0.018357989410994084),
 ('Company', 0.016595436599855982),
 ('WSGS reservoir', 0.003914409965170883),
 ('Well type', 0.0007

So the `Total Vertical Depth` of a well is the most important feature in predicting the first 18 months of a wells oil production followed by the `Surface hole longitude`, `Bottom hole longitude` and the `Total Proppant`. If we assume that TVD and location are considered geologic factors then geology has more influence in predicting production than the completion design of these Turner Sandstone wells in the Powder River Basin. This agrees with the conclusions of [RI-77](http://sales.wsgs.wyo.gov/influences-on-oil-and-natural-gas-production-from-the-wall-creek-and-turner-sandstone-reservoirs-powder-river-basin-wyoming-2019/?ctk=2a2030c2-bb60-4a4d-aa30-14502a0aee7c) that the complex geology of the reservoir should be considered when designing these wells. 

It's interesting seeing that the `Company` feature and the `WSGS reservoir` have some of the least importance in predicting the first 18 months of production. There you have it, we took a messy spreadsheet, munged the data in it, and then used the munged data to come to the same conclusion as the report in about 40 lines of Python. 
### Yay for reproducibility!

This notebook is licensed as CC-BY, use and share to your hearts content.